### 猪价及供给端的数据清洗整合

1. 选取数据并进行清洗整合：几个变量处于不同数量级，不同年月日。主要目的是将他们进行日期的统一化和数据正则化。

5大数据：
    - 猪价：日度数据，周末为空白值
    - 能繁存栏：官方给定数据为季度数据，第三方数据一般为月度数据
    - 出栏：官方给定数据为季度数据，第三方的抽样调查为月度数据
    - 屠宰：官方给定数据为季度或月度数据，三方抽样调查为月度数据
    - 出栏均重

能繁、出栏、屠宰、出栏均重为供给端数据，能繁会与PSY以及育肥成活率决定10个月后的市场产能；而出栏有可能并不是全部流入市场，可能在养殖端进行买卖给其他农户做二次育肥；屠宰量为当期流入市场的猪量，和宰后均重共同决定当期流入市场的真正供给量。

变量间理论关系：能繁在后移10个月后应与出栏、屠宰三者呈正相关关系；当屠宰和出栏呈负相关关系时证明出栏的猪并没有完全流入市场，而是有部分流入养殖户手中，若是用于二次育肥，那么后期1-2个月的宰后均重和出栏均重应该有明显增长；当屠宰和出栏关系正常而能繁和猪价关系不正常时要看当期PSY和MSY是否有变化，PSY和MSY的变化可能抹平能繁的异常变化。



In [20]:
# 1.1 猪价季节性特征

# 加载数据包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
# 载入数据，猪价采用钢联数据库提供日度猪价数据
data_price = pd.read_excel("database/GL_20230227.xlsx", sheet_name="外三元现货价（2013年起）")

In [30]:
# 转换行列名
data_price.columns = ["日期", "价格"]
data_price = data_price.drop(index=0)

In [31]:
data_price["日期"] = pd.to_datetime(data_price["日期"])
data_price = data_price.set_index("日期")


In [32]:
data_monthly = data_price.groupby([data_price.index.year, data_price.index.month]).mean()


In [39]:
data_monthly.tail()

价格
日期   日期           
2022 10  26.387143
     11  25.192273
     12  20.177727
2023 1   15.385909
     2   14.494286

In [45]:
# 获取年份和月份列表
years = data_price.index.year.unique()
months = data_price.index.month.unique()

In [46]:
months

Int64Index([2, 1, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3], dtype='int64', name='日期')

In [ ]:

# 绘制折线图
for year in years:
    for month in months:
        # 从数据中获取该年份和月份的销售额
        sales = data_monthly.loc[(year, month), "价格"]
        # 绘制折线图
        plt.plot(sales.index, sales, label=f'{year}-{month}')

# 添加标签和标题
plt.xlabel('Month')
plt.ylabel('Sales')
plt.title('Sales Trends')

# 设置x轴标签的刻度
x_ticks = [f'{year}-{month}' for year in years for month in months]
plt.xticks(range(len(x_ticks)), x_ticks, rotation=45, ha='right')

# 添加图例
plt.legend()

# 显示图形
plt.show()
